<a href="https://colab.research.google.com/github/InduwaraGayashan001/Generative-AI/blob/main/Falcon_Multi-Doc_Retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!pip install sentence_transformers
!pip -q install langchain tiktoken chromadb pypdf transformers
!pip -q install accelerate bitsandbytes sentencepiece Xformers
!pip -q install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

# Load Data

In [6]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader, DirectoryLoader

In [7]:
!wget https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip -O stock_market_june_2023.zip
!unzip stock_market_june_2023.zip

--2025-06-22 16:21:41--  https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip [following]
--2025-06-22 16:21:41--  https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10638 (10K) [application/zip]
Saving to: ‘stock_market_june_2023.zip’

stock_market_june_2 100%[===================>]  10.39K  --.-KB/s    in 0s      

2025-06-22 16:21:42 (114 MB/s) - ‘stock_market_june_2023.zip’ saved [10638/10638]

In [7]:
loader = DirectoryLoader('./stock_market_june_2023', glob='**/*.txt', loader_cls=TextLoader)
documents = loader.load()

In [8]:
documents

[Document(metadata={'source': 'stock_market_june_2023/Shopify.txt'}, page_content='Shopify (NYSE: SHOP) has 55% increase in its shares so far in 2023. Despite ongoing price fluctuations, Shopify’s business and growth trajectory are moving upward.\nIn the seven years since its initial public offering, Shopify has been profitable in five, a commendable track record in an industry known for widespread unprofitability. The company has weathered economic volatility, including the challenges posed by the pandemic, by streamlining its focus on delivering market-leading software and solutions to empower its merchant clients.\n\nTo adapt to the changing landscape, Shopify implemented significant changes, including workforce layoffs of approximately 20% and selling its logistics business to Flexport. Although these changes incur near-term expenses, such as restructuring and impairment charges, they will ultimately reduce overhead costs and make Shopify more asset-light in the long run.\n\nIn Q1,

# Split into chunks

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

In [11]:
len(texts)

49

In [12]:
texts[2]

Document(metadata={'source': 'stock_market_june_2023/Shopify.txt'}, page_content='To adapt to the changing landscape, Shopify implemented significant changes, including workforce layoffs of approximately 20% and selling its logistics business to Flexport. Although these changes incur near-term expenses, such as restructuring and impairment charges, they will ultimately reduce overhead costs and make Shopify more asset-light in the long run.')

# Load the LLM

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import transformers

model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

# Load Embeddings

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings

llm = HuggingFacePipeline(pipeline=pipeline)
model = "intfloat/e5-large-v2"
hf = HuggingFaceEmbeddings(model_name=model)

# Intialize Chromadb

In [10]:
from langchain.vectorstores import Chroma
persist_directory = 'db'

embedding =hf

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [11]:
vectordb.persist()

vectordb = None

vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=hf)


/tmp/ipython-input-11-1530081794.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()
/tmp/ipython-input-11-1530081794.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory,


# Calling LLM

In [12]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [13]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [14]:
query = " Could you please enumerate the companies that have been highlighted to their potential stock growth"
llm_response = qa_chain(query)
process_llm_response(llm_response)

/tmp/ipython-input-14-1064061156.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm_response = qa_chain(query)
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Arbor Metals Corp.
Tesla Inc.
Kiplin Metals Inc.
Microsoft
NVIDIA
Shopify
Intuitive Surgical
MercadoLibre
Growth stocks like these are ideal for investors seeking short-term gains and long-term resilience. These companies are primed to withstand future market fluctuations through innovation, market opportunities, embedding in recession-resistant sectors, and committing to diversifying their portfolios.

Future-proofing your portfolio with growth stocks isn’t just about singling out products and industries that will always be in demand. 
It’s also about identifying companies engineered from inception to adapt to industry shifts. Microsoft (NASDAQ: MSFT) fits these criteria like a glove.

Institutional ETFs assembled by Goldman Sachs and Vanguard rely on growth stocks to drive their price in all market conditions, and your por